In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')
#print(test)
print(df.head(5))
outid = test['review_id'].values # dataframe to numpy
#print(outid.T)
#print(len(outid))
train = df

   review_id  business_id  user_id  \
0       3223         2055     2533   
1       9938         4165     6371   
2       7123          869     4929   
3       3601         1603     2789   
4       3948         2347     1245   

                                                text        date  stars  
0  Sometimes things happen, and when they do this...  2010-12-30      5  
1  I know Kerrie through my networking and we ben...  2011-04-26      5  
2  Love their pizza!!!\r\nVery fresh. Their canno...  2012-09-28      5  
3  Being from NJ I am always on the prowl for my ...  2009-06-07      4  
4  We have tried this spot a few times and each v...  2011-02-20      4  


In [2]:
from sklearn.svm import LinearSVR
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = df.drop(['review_id','business_id','user_id','date'], axis=1) 

'''
vectorizer = CountVectorizer()
vectorizer.fit(train['text'])
'''

vectorizer = TfidfVectorizer(ngram_range=[1,3],max_df = 2000)#,min_df = 0.01)#,stop_words="english",min_df = 0.009) 
vectorizer.fit(train['text'])

X_train = vectorizer.transform(train['text'])
X_test = vectorizer.transform(test['text'])
y_train = train['stars'].as_matrix()

clf = LinearSVR() # default = 1000
clf = clf.fit(X_train, y_train)
pred_y = clf.predict(X_test)

print(clf.score(X_train, y_train))

for i in range(0,len(pred_y)):
    if(pred_y[i]>5):
        pred_y[i] = 5.0
    elif(pred_y[i]<1):
        pred_y[i] = 1.0
    else:
        pred_y[i] = round(pred_y[i],2)
        
output = np.vstack((outid,pred_y)).T
output = pd.DataFrame(output,columns=['user_id','stars'])
output['user_id'] = output['user_id'].astype(int)
np.savetxt("L_SVR2.csv", output, fmt='%i, %1.4f', delimiter=",")


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


0.907307738700597
